# A new path: /echo

Now we're going to write a new path: `/echo`.

Our goal is the following:

- when invoking `/datetime/v1/echo` the API will return 
  the current datetime;
- the response is enveloped in the following example json object:

```
{ "timestamp": "2019-12-25T00:00:00Z" }
```

- the status code for a successful response is `200`

### Exercise: write /echo specs

Edit the [ex-04-02-echo-ok.yaml](/edit/notebooks/oas3/ex-04-02-echo.yaml]) and write the `echo` specifications:

1- define the new `Datetime` schema to be used in the response;

2- define the new `/echo` path supporting the `get` method and:

  * always write proper `summary` and `description` fields
  
3- `get /echo` possible responses are:

  * `200` returning a `Datetime` in json format, with a complete `example` for mocks
  * `503` returning a `problem+json`
  

4- don't forget `operationId` !

Hint: feel free to reuse as much yaml code as possible.

### Exercise: test /echo mocks

Run your spec [in the terminal](/terminals/1) and check that it properly returns the mock objects.

Use 

```
connexion run --mock all /code/notebooks/oas3/ex-04-02-echo-ok.yaml
```

In [ ]:
# Use this cell to test the output
!curl http://localhost:5000/datetime/v1/echo -vk

## Request parameters

OAS3 allows to specify parameters both in `query` and in `headers`.

Interoperable APIs should follow simple rules when using parameters:

- define parameter conventions for common patterns, eg: limit, offset;
- use standard HTTP headers when possible.

As many APIs implement pagination, it's a good think to converge to a common
set of parameters to use. Our choice is:

- limit: max number of entries to retrieve
- offset: the number of entries to skip in a paginated request
- cursor: an identifier (cursor) of the first entry to be returned  
  [Slack APIs provide an example of cursor-based pagination](https://api.slack.com/docs/pagination)

## Adding request parameters to `/echo`

Request parameters can be defined in 

```
components:
  parameters:
    a_sample_parameter:
      ...
```

In [3]:
# Here's the `limit` parameter definition, including
# - it's a `query` parameter
# - its name
# - its schema
print(show_component('https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml#/parameters/limit'))

description: How many items to return at one time (max 100)
in: query
name: limit
schema:
  format: int32
  type: integer



### Exercise: adding parameters to `/echo`

Let's add a `tz` parameter to `/echo`:

- create a `#/components/schemas/Timezone` schema definition. Valid values are:

  * 'Europe/Rome'
  * 'UTC' (default)
  * 'Asia/Calcutta'
  
- create a `#/components/parameters/tz` parameter definition;
- add the `tz` query parameter to `get /echo` path checking
  the [official OAS 3.0.2 documentation](https://github.com/OAI/OpenAPI-Specification/blob/master/versions/3.0.2.md#examples)
- add a `400 Bad Request` response in case the timezone is not in the list

Finally, check that you can run the spec.

```
connexion run --mock all /code/notebooks/oas3/ex-04-02-echo-ok.yaml
```

In [4]:
# Use this cell to test your api

### Implement get_status

Let's implement the `get_echo` in [api.py](/edit/notebooks/oas3/api.py) function that:

- takes the `tz` parameter, which defaults to `UTC`;
- returns a `Datetime` object in its current timezone.

In [ ]:
# hint: get the timezone from 
!pip install pytz

In [ ]:
import pytz 
tz = pytz.timezone('Europe/Rome')
print(tz.name)

In [11]:
# No timezones in Netherland :)
'Netherland' in pytz.all_timezones

False

In [ ]:
# connexion provides a predefined problem object
from connexion import problem

    
# Exercise: write a get_status() returning a successful response to problem.
help(problem)

In [13]:
def get_echo(tz='UTC'):
    if tz not in pytz.all_timezones:
        return problem(
            status=400,
            title="Bad Timezone",
            detail="The specified timezone is not valid",
            ext={"valid_timezones": pytz.all_timezones}
        )
    d = datetime.now(tz=pytz.timezone(tz))
    return {"timestamp": str(d).replace('+00:00', 'Z')}
    

Now  [run the spec in a terminal](/terminals/1) using

```
cd /code/notebooks/oas3/
connexion run /code/notebooks/oas3/ex-04-02-echo-ok.yaml
```

play a bit with the [Swagger UI](https://TODO)

and try making a request!


In [ ]:
!curl http://localhost:5000/datetime/v1/echo -kv

!curl http://localhost:5000/datetime/v1/echo?tz=Europe/Rome -kv


### Returning errors.

Our api.get_status implementation always returns `200 OK`, but in the real world APIs could
return different kind of errors.

An interoperable API should:

- fail fast, avoiding that application errors result in stuck connections;
- implement a clean error semantic.

In our Service Management framework we expect that:

- if the Service is unavailable, we must return `503 Service Unavailable` http status
- we must return the `Retry-After` header specifying the number of seconds
  when to retry.

TODO: ADD CIRCUIT_BREAKER IMAGE

To implement this we must:

1. add the returned headers in the OAS3 interface;
2. pass the headers to the flask Response


### Exercise

Modify the OAS3 spec in [ex-04-01-headers.yaml](/edit/notebooks/oas3/ex-04-01-headers-ok.yaml) and:

- add a `503` response to the `/status` path;
- when a `503` is returned, the `retry-after` header is returned.

Hint:  you can define a header in `components/headers` like that:

```
components:
  headers:
    Retry-After:
      description: |-
        Retry contacting the endpoint *at least* after seconds.
        See https://tools.ietf.org/html/rfc7231#section-7.1.3
    schema:
      format: int32
      type: integer

```


Or just `$ref` the `Retry-After` defined in https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml#/headers/Retry-After

Modify [api.py:get_status](/edit/notebooks/oas3/api.py) such that:

- returns a `503` on 20% of the requests;
- when a `503` is returned, the `retry-after` header is returned.


In [ ]:
from random import randint


def get_status():
    p = randint(1, 5)
    if p == 5:
        return problem(
            status=503,
            title="Service Temporarily Unavailable",
            detail="Retry after the number of seconds specified in the the Retry-After header.",
            headers={'Retry-After': str(p)}
        )
    return problem(
        status=200,
        title="OK",
        detail="So far so good."
    )

## Reusing default responses

As `503` is a quite recurring response, it's worth to define it in a reusable yaml file,
so that every path can reuse it.


### Exercise: reusable responses

In the following exercise you should edit [ex-04-01-headers.yaml](/edit/notebooks/oas3/ex-04-01-headers-ok.yaml) and:

- move the `503` response from the `/status` path definition to the `components/responses` one, eg.

```
components:
  responses:
    503ServiceUnavailable:
      ...
```

- reference `#/components/responses/503ServiceUnavailable` in the `/status` path

Your new file is semantically equivalent to the previous one: check that you can 
nicely `connexion run` your file!

```
connexion run /code/notebooks/oas3/ex-04-01-headers.yaml
```
